In [ ]:
# This takes a dataset, prepares the data, splits it, tries out different ML models,
# picks the best one based on test data, checks it on validation data, saves the pipeline,
# and maybe tries out a web app

# (In the style of Chpt. 2 exercises, p. 84)

In [ ]:
### Read in data

In [ ]:
### Pre-process the data

In [ ]:
### Split into training and validation datasets

In [ ]:
### Try individual ML models with a grid search for each

In [ ]:
### Which model appears to perform best?

In [ ]:
### Check performance of all models on validation data. 
### Is the best model based on the training data also the best based on the validation data?

In [ ]:
### Save the best pipeline

In [ ]:
### Try a web app (may need to read bookmarked websites)